# Tahoe Forest Health Threshold Analysis
* Mason Bindl, mbindl@trpa.gov
* Andrew McClary, amcclary@trpa.gov

##### Links
* https://caregionalresourcekits.org//sierra.html#forest_res

### Setup

In [2]:
# import packages
from utils import *
import os
import arcpy
from arcpy.sa import *
from arcpy.ia import *
from arcpy.ddd import *
import pandas as pd
from pathlib import Path
from functools import reduce
# setup workspace variables
arcpy.env.workspace = r"F:\\GIS\\PROJECTS\\ForestHealth_Intiative\\ThresholdUpdate\\Data\\ForestHealth_ThresholdUpdate.gdb"
arcpy.env.overwriteOutput = True

#format paths
sdeBase   = Path("F:\GIS\DB_CONNECT\Vector.sde")
workspace = Path("F:\GIS\PROJECTS\ForestHealth_Intiative\ThresholdUpdate\Data\ForestHealth_ThresholdUpdate.gdb")
downloads = Path("F:\GIS\PROJECTS\ForestHealth_Intiative\ThresholdUpdate\Data\Download\SNV_RRK")
output    = Path("F:\GIS\PROJECTS\ForestHealth_Intiative\ThresholdUpdate\AnalysisProduct")

# base layers
trpa_boundary       = sdeBase / "SDE.Jurisdictions\SDE.TRPA_bdy"
mgmt_areas          = sdeBase / "SDE.Jurisdictions\SDE.ForestManagementZone_USFS"

# raw raster data downloaded from SNVRRK
veg_type            = downloads / "CWHR_vegetation\RRK_Fveg_WHRtype_2023Apr_4regions_v2.tif"
stand_density       = downloads / "TPA_2021_30m\TPA_2021_30m.tif"
basal_area          = downloads / "BASATOT_2021_30m\BASATOT_2021_30m.tif"
seral_stage         = downloads / "SeralStage_EML_2021\SeralStage_EML_2021.tif"
canopy_cover        = downloads / "CFO_CanopyCover2020Summer\CFO_CanopyCover2020Summer.tif"
fire_prob_low       = downloads / "ProbabilityLowFireSev_202308\ProbabilityLowFireSev_202308.tif"
fire_prob_moderate  = downloads / "ProbabilityMedFireSev_202308\ProbabilityModerateFireSev_202308.tif"
fire_prob_high      = downloads / "ProbabilityHighFireSev_202308\ProbabilityHighFireSev_202308.tif"

In [ ]:
# list datasets in workspace
feature_data = arcpy.ListFeatureClasses()
raster_data  = arcpy.ListRasters()
table_data   = arcpy.ListTables()
# print feature classes, then rasters, then tables
print("Feature Classes:")
for fc in feature_data:
    print(fc)
print("\nRasters:")
for raster in raster_data:
    print(raster)
print("\nTables:")
for table in table_data:
    print(table)

## Extract to Tahoe

In [4]:
# save a verions of each input tif at full extent and extract the extent to Tahoe
# create a list of the rasters to be clipped
raster_list = [
    veg_type,
    stand_density,
    basal_area,
    seral_stage,
    canopy_cover,
    fire_prob_low,
    fire_prob_moderate,
    fire_prob_high
]
# names of the rasters to be clipped
raster_names = [
    "veg_type_sierra",
    "stand_density_sierra",
    "basal_area_sierra",
    "seral_stage_sierra",
    "canopy_cover_sierra",
    "fire_prob_low_sierra",
    "fire_prob_moderate_sierra",
    "fire_prob_high_sierra"
]

# save a version of each input tif at full extent
for raster, name in zip(raster_list, raster_names):
    # save a version of each input tif at full extent
    out_raster = f"{name}"
    arcpy.sa.Raster(str(raster)).save(out_raster)
    print(f"Saved {name} to full extent")
# extract by mask and add _tahoe to the name
for raster, name in zip(raster_list, raster_names):
    # extract by mask
    out_extract = extract_by_mask_to_tahoe_extent(raster, f"{name}_tahoe")
    print(f"Extracted {name} to Tahoe")

Saved veg_type_sierra to full extent
Saved stand_density_sierra to full extent
Saved basal_area_sierra to full extent
Saved seral_stage_sierra to full extent
Saved canopy_cover_sierra to full extent
Saved fire_prob_low_sierra to full extent


RuntimeError: ERROR 000732: Input Raster: Dataset F:\GIS\PROJECTS\ForestHealth_Intiative\ThresholdUpdate\Data\Download\SNV_RRK\ProbabilityMedFireSev_202308\ProbabilityModerateFireSev_202308.tif does not exist or is not supported

## Stand Density

#### Stand Density - Clip to Tahoe & convert to integer

In [ ]:
# create integer raster for stand density
int_raster = Int("stand_density_sierra_tahoe")
int_raster.save("stand_density_sierra_tahoe_int")
print("Converted stand density to integer raster")
# create integer raster for basal area
int_raster = Int("basal_area_sierra_tahoe")
int_raster.save("basal_area_sierra_tahoe_int")
print("Converted basal area to integer raster")

# build attribute table
arcpy.BuildRasterAttributeTable_management("stand_density_sierra_tahoe_int", "Overwrite")
arcpy.BuildRasterAttributeTable_management("basal_area_sierra_tahoe_int", "Overwrite")

In [ ]:
# function to combine stand density and vegetation type rasters
def combine_stand_density_and_veg_type(
    stand_density_raster,
    veg_type_raster,
    output_workspace,
    combined_raster_name="StandDensity_VegType_Combined",
    lookup_table_name="veg_lookup_table"
):
    """Combines stand density and vegetation type rasters into one raster with attributes."""
    
    arcpy.env.workspace = output_workspace
    arcpy.env.overwriteOutput = True
    
    # Paths
    combined_raster = os.path.join(output_workspace, combined_raster_name)
    veg_lookup_table = os.path.join(output_workspace, lookup_table_name)
    
    # In-memory paths
    veg_type_resampled = r"in_memory\VegType_Resampled"
    veg_type_clipped = r"in_memory\VegType_Clipped"
    stand_density_projected = r"in_memory\StandDensity_Projected"
    veg_type_projected = r"in_memory\VegType_Projected"
    
    # Target spatial reference (NAD83 UTM Zone 10N)
    target_sr = arcpy.SpatialReference(26910)
        
    # Create Lookup table
    if not arcpy.Exists(veg_lookup_table):
        print("Veg lookup table not found. Creating from vegetation raster attribute table...")
        arcpy.conversion.TableToTable(veg_type_raster, output_workspace, lookup_table_name)
    else:
        print("Found existing vegetation lookup table.")
    
    # Project rasters if needed
    stand_density_raster_proj = project_if_needed(stand_density_raster, stand_density_projected, target_sr)
    veg_type_raster_proj      = project_if_needed(veg_type_raster, veg_type_projected, target_sr)
    
    # Resample vegetation
    print("Resampling vegetation raster to match stand density...")
    stand_density_desc = arcpy.Describe(stand_density_raster_proj)
    cell_size = stand_density_desc.meanCellWidth
    extent = stand_density_desc.exten
    arcpy.management.Resample(veg_type_raster_proj, veg_type_resampled, cell_size, "NEAREST")
    
    # Clip vegetation to stand density extent
    print("Clipping resampled vegetation raster...")
    arcpy.management.Clip(
        veg_type_resampled, 
        f"{extent.XMin} {extent.YMin} {extent.XMax} {extent.YMax}",
        veg_type_clipped,
        in_template_dataset=None,
        nodata_value="0",
        clipping_geometry="NONE",
        maintain_clipping_extent="MAINTAIN_EXTENT"
    )
    
    # Combine rasters
    print("Combining stand density and vegetation rasters...")
    out_combine = arcpy.sa.Combine([stand_density_raster_proj, veg_type_clipped])
    out_combine.save(combined_raster)
    
    # Join veg type names
    print("Joining vegetation type names to combined raster...")
    combine_fields = [f.name for f in arcpy.ListFields(combined_raster)]
    print(f"Fields in combined raster: {combine_fields}")
    
    base_name = os.path.basename(veg_type_raster)
    veg_field_guess = os.path.splitext(base_name)[0]
    
    if veg_field_guess not in combine_fields:
        possible_fields = [f for f in combine_fields if f not in ['OBJECTID', 'Value', 'Count']]
        if possible_fields:
            veg_value_field = possible_fields[1]  # Assuming the second field is the vegetation value
            print(f"Auto-detected vegetation value field: {veg_value_field}")
        else:
            raise Exception("Cannot find vegetation field in combined raster.")
    else:
        veg_value_field = veg_field_guess
    
    if "Value" not in [f.name for f in arcpy.ListFields(veg_lookup_table)]:
        raise ValueError("The lookup table does not contain a 'VALUE' field for joining.")
    
    arcpy.management.JoinField(
        combined_raster, 
        veg_value_field,  # <-- THIS IS KEY
        veg_lookup_table, 
        "Value", 
        ["TRPA_VegType"]  
    )    
    # change final field names
    arcpy.management.AlterField(combined_raster, veg_value_field, "VegTypeCode", "VegTypeCode")
    arcpy.management.AlterField(combined_raster, "StandDensity_Projected", "StandDensity", "StandDensity")

    add_acres(combined_raster)
    print(f"Combined raster is ready at: {combined_raster}")
    
    # Clean up
    arcpy.management.Delete(veg_type_resampled)
    arcpy.management.Delete(veg_type_clipped)
    arcpy.management.Delete(stand_density_projected)
    arcpy.management.Delete(veg_type_projected)
    print("Cleaned up in-memory data.")

In [ ]:
combine_stand_density_and_veg_type(
    stand_density_raster="StandDensity_Tahoe_SNRRK_int",
    veg_type_raster= "VegType_30m",
    output_workspace=arcpy.env.workspace,
    combined_raster_name="StandDensity_VegType_Combined",
    lookup_table_name="veg_lookup_table"
)

## Composition & Age

In [ ]:
seral_stage         = downloads / "SeralStage_EML_2021\SeralStage_EML_2021.tif"
canopy_cover        = downloads / "CFO_CanopyCover2020Summer\CFO_CanopyCover2020Summer.tif"

Seral   = Raster(str(seral_stage))
Canopy  = Raster(str(canopy_cover))

# dictionay of lookup values
lookup_dict = {1:"Early Seral Open", 
               2:"Early Seral Closed", 
               3:"Mid Seral Open", 
               4:"Mid Seral Closed", 
               5:"Late Seral Open", 
               6:"Late Seral Closed"}

# Use the Con function to set the output values 1=Early Seral Open, etc..
SeralCanopy40 =         Con(((Raster(Seral) == 1) & (Raster(Canopy) <40)),1, 
                        Con(((Raster(Seral) == 1) & (Raster(Canopy)>=40)),2,
                        Con(((Raster(Seral) == 2) & (Raster(Canopy) <40)),3, 
                        Con(((Raster(Seral) == 2) & (Raster(Canopy)>=40)),4,
                        Con(((Raster(Seral) == 3) & (Raster(Canopy) <40)),5, 
                        Con(((Raster(Seral) == 3) & (Raster(Canopy)>=40)),6))))))

# Use the Con function to set the output values 
SeralCanopy60 =         Con(((Raster(Seral) == 1) & (Raster(Canopy) <60)),1, 
                        Con(((Raster(Seral) == 1) & (Raster(Canopy)>=60)),2,
                        Con(((Raster(Seral) == 2) & (Raster(Canopy) <60)),3, 
                        Con(((Raster(Seral) == 2) & (Raster(Canopy)>=60)),4,
                        Con(((Raster(Seral) == 3) & (Raster(Canopy) <60)),5, 
                        Con(((Raster(Seral) == 3) & (Raster(Canopy)>=60)),6))))))

# Set output raster
output_raster40_SNV = "SeralStage_CanopyClass_40prcnt_30m_Sierra_SNVRRK"
output_raster60_SNV = "SeralStage_CanopyClass_60prcnt_30m_Sierra_SNVRRK"

# Save the output raster to the geodatabase
SeralCanopy40.save(output_raster40_SNV)
SeralCanopy60.save(output_raster60_SNV)

# build an attribute table
arcpy.BuildRasterAttributeTable_management(output_raster40_SNV, "Overwrite")
arcpy.BuildRasterAttributeTable_management(output_raster60_SNV, "Overwrite")
# calc Acres field
add_acres(output_raster40_SNV)
add_acres(output_raster60_SNV)
# add and calc category field
add_category(output_raster40_SNV, lookup_dict)
add_category(output_raster60_SNV, lookup_dict)

# extract by mask to Tahoe extent
out_raster = ExtractByMask(
    in_raster=output_raster40_SNV,
    in_mask_data=r"F:\GIS\DB_CONNECT\Vector.sde\sde.SDE.Jurisdictions\sde.SDE.TRPA_bdy",
    extraction_area="INSIDE",
    analysis_extent='-214749.813147473 -338358.008101731 228897.27559438 457005.517540967 PROJCS["NAD_1983_California_Teale_Albers",GEOGCS["GCS_North_American_1983",DATUM["D_North_American_1983",SPHEROID["GRS_1980",6378137.0,298.257222101]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]],PROJECTION["Albers"],PARAMETER["False_Easting",0.0],PARAMETER["False_Northing",-4000000.0],PARAMETER["Central_Meridian",-120.0],PARAMETER["Standard_Parallel_1",34.0],PARAMETER["Standard_Parallel_2",40.5],PARAMETER["Latitude_Of_Origin",0.0],UNIT["Meter",1.0]]'
)

# extract by mask to Tahoe extent
out_raster = ExtractByMask(
    in_raster=output_raster60_SNV,
    in_mask_data=r"F:\GIS\DB_CONNECT\Vector.sde\sde.SDE.Jurisdictions\sde.SDE.TRPA_bdy",
    extraction_area="INSIDE",
    analysis_extent='-214749.813147473 -338358.008101731 228897.27559438 457005.517540967 PROJCS["NAD_1983_California_Teale_Albers",GEOGCS["GCS_North_American_1983",DATUM["D_North_American_1983",SPHEROID["GRS_1980",6378137.0,298.257222101]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]],PROJECTION["Albers"],PARAMETER["False_Easting",0.0],PARAMETER["False_Northing",-4000000.0],PARAMETER["Central_Meridian",-120.0],PARAMETER["Standard_Parallel_1",34.0],PARAMETER["Standard_Parallel_2",40.5],PARAMETER["Latitude_Of_Origin",0.0],UNIT["Meter",1.0]]'
)
# save.
out_raster.save("SeralStage_CanopyClass_40prcnt_30m_Tahoe_SNVRRK")
out_raster.save("SeralStage_CanopyClass_60prcnt_30m_Tahoe_SNVRRK")

output_raster40_Tahoe = "SeralStage_CanopyClass_40prcnt_30m_Tahoe_SNVRRK"
output_raster60_Tahoe = "SeralStage_CanopyClass_60prcnt_30m_Tahoe_SNVRRK"

# build an attribute table
arcpy.BuildRasterAttributeTable_management(output_raster40_Tahoe, "Overwrite")
arcpy.BuildRasterAttributeTable_management(output_raster60_Tahoe, "Overwrite")
# calc Acres field
add_acres(output_raster40_Tahoe)
add_acres(output_raster60_Tahoe)
# add and calc category field
add_category(output_raster40_Tahoe, lookup_dict)
add_category(output_raster60_Tahoe, lookup_dict)
# get raster attribute table as a pandas dataframe
arcpy.conversion.ExportTable(
    in_table=output_raster40_Tahoe,
    out_table= str(output / "SeralStage_CanopyClass_40prcnt_30m_Tahoe_SNVRRK.csv")
)
arcpy.conversion.ExportTable(
    in_table=output_raster60_Tahoe,
    out_table= str(output / "SeralStage_CanopyClass_60prcnt_30m_Tahoe_SNVRRK.csv")
)
# export to csv
arcpy.conversion.ExportTable(
    in_table=output_raster40_SNV,
    out_table= str(output / "SeralStage_CanopyClass_40prcnt_30m_Sierra_SNVRRK.csv")
)
arcpy.conversion.ExportTable(
    in_table=output_raster60_SNV,
    out_table= str(output / "SeralStage_CanopyClass_60prcnt_30m_Sierra_SNVRRK.csv")
)

## Functional Fire

##### Dominant Flame Length Class Analysis

In [ ]:
# probability of fire severity 2023
fire_prob_high     = Raster(str(downloads / "ProbabilityHighFireSev_202308/ProbabilityHighFireSev_202308.tif"))
fire_prob_moderate = Raster(str(downloads / "ProbabilityModerateFireSev_202308/ProbabilityModerateFireSev_202308.tif"))
fire_prob_low      = Raster(str(downloads / "ProbabilityLowFireSev_202308/ProbabilityLowFireSev_202308.tif"))

# extract by mask to Tahoe extent
extract_by_mask_to_tahoe_extent(fire_prob_high, "FunctionalFire_HighSeverityProbabiliy_Tahoe_SNVRRK")
extract_by_mask_to_tahoe_extent(fire_prob_moderate, "FunctionalFire_ModerateSeverityProbability_Tahoe_SNVRRK")
extract_by_mask_to_tahoe_extent(fire_prob_low, "FunctionalFire_LowSeverityProbability_Tahoe_SNVRRK")

##### Dominant Class Analysis

In [ ]:
from arcpy.sa import Con, Int, Raster

# Load the raw fire severity rasters
highsev_tahoe = Raster("FunctionalFire_HighSeverityProbabiliy_Tahoe_SNVRRK")
modsev_tahoe = Raster("FunctionalFire_ModerateSeverityProbability_Tahoe_SNVRRK")
lowsev_tahoe = Raster("FunctionalFire_LowSeverityProbability_Tahoe_SNVRRK")

# Calculate dominant class
# Use Con and logical operators to determine max
dominant_class = Con(
    (lowsev_tahoe >= modsev_tahoe) & (lowsev_tahoe >= highsev_tahoe), 1,
    Con((modsev_tahoe >= lowsev_tahoe) & (modsev_tahoe >= highsev_tahoe), 2, 3)
)

# Convert to integer raster
dominant_class_int = Raster(Int(dominant_class))

# Save the output raster to the geodatabase
dominant_class_int.save("FunctionalFire_DominantClass_Tahoe_SNVRRK")
print("Exported raster: FunctionalFire_DominantClass_Tahoe_SNVRRK")

# Build an attribute table
arcpy.BuildRasterAttributeTable_management(dominant_class_int, "Overwrite")

# Calculate Acres field
add_acres(dominant_class_int)

# Save the updated raster
dominant_class_int.save("FunctionalFire_DominantClass_Tahoe_SNVRRK")
print("Updated raster: FunctionalFire_DominantClass_Tahoe_SNVRRK")

In [ ]:
# export to feature class
arcpy.RasterToPolygon_conversion(
    in_raster="FunctionalFire_DominantClass_Tahoe_SNVRRK",
    out_polygon_features="FunctionalFire_DominantClass_Tahoe_SNVRRK_Polygon",
    simplify="NO_SIMPLIFY",
    raster_field="Value"
)
# add acres field
arcpy.management.AddField("FunctionalFire_DominantClass_Tahoe_SNVRRK_Polygon", "Acres", "DOUBLE")
# calculate acres field
arcpy.management.CalculateField("FunctionalFire_DominantClass_Tahoe_SNVRRK_Polygon", "Acres", "!shape.area@acres!", "PYTHON3")

# identiy with management zones
arcpy.analysis.Identity(
    in_features="FunctionalFire_DominantClass_Tahoe_SNVRRK_Polygon",
    identity_features=str(mgmt_areas),
    out_feature_class="ID_MGMTzones_FunctionalFire_DominantClass_Tahoe_SNVRRK",
)

# calculate acres
arcpy.management.CalculateField("ID_MGMTzones_FunctionalFire_DominantClass_Tahoe_SNVRRK", "Acres", "!shape.area@acres!", "PYTHON3")